# Run Union Workflows on a Jupyter Notebook

First create a Union Serverless account:
👉 https://signup.union.ai/

First install the dependencies:

In [1]:
%%capture
%pip install "git+https://github.com/flyteorg/flytekit@3e3ec17ea" union "pydantic>2"

Then, login to Union on this notebook session:

In [ ]:
!union create login --auth device-flow --serverless

## Create some data

Let's create some toy data:

In [6]:
import flytekit as fk
import sys


image = fk.ImageSpec(
    name="jupyter-notebook-workshop",
    packages=["git+https://github.com/flyteorg/flytekit@3e3ec17ea"],
    apt_packages=["git"],
    python_version=f"{sys.version_info.major}.{sys.version_info.minor}",
)


task = fk.task(container_image=image)


@task
def create_dataset(n: int) -> list[int]:
    return [*range(n)]


@fk.workflow
def dataset_wf(n: int = 100) -> list[int]:
    return create_dataset(n)

## Running on Union

Let's define a `UnionRemote` object:

In [7]:
from union.remote import UnionRemote
from union.ucimage._image_builder import _register_union_image_builder

_register_union_image_builder() 
remote = UnionRemote()

13:24:31.751523 WARNING  remote.py:286 - Jupyter notebook and interactive task  
                         support is still alpha.

Then, we can execute the workflow like so:

In [8]:
exe = remote.execute(dataset_wf, inputs={"n": 50})
print(exe.execution_url)

Image cr.union.ai/jupyter-notebook-workshop:ntYVSE9agvQyq_KMWZJyLw found. Skip building.
https://serverless.union.ai/org/cosmicbboy/projects/flytesnacks/domains/development/executions/ajmgkph48dwglmm9bp2d


We can fetch the output of the workflow back into our notebook:

In [9]:
# Wait for the task to complete
exe = exe.wait(poll_interval=1)

# Print the outputs
print(exe.outputs)

{'o0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]}


## Compute the mean

We can then pass the data back into another workflow that computes the mean:

In [13]:
@task
def compute_mean(data: list[int]) -> float:
    return sum(data) / len(data)


@fk.workflow
def mean_wf(data: list[int]) -> float:
    return compute_mean(data)

Run the workflow:

In [14]:
# Execute the workflow
data, *_ = exe.outputs.values()
exe = remote.execute(mean_wf, inputs={"data": data})
print(exe.execution_url)

https://serverless.union.ai/org/cosmicbboy/projects/flytesnacks/domains/development/executions/acvd782grmn4sdf2hnw8


In [15]:
# Wait for the task to complete
exe = exe.wait(poll_interval=1)

# Print the outputs
print(exe.outputs)

{'o0': 24.5}


And that's it! You can now iterate on your workflows and tasks directly inside a Jupyter Notebook 🚀